# 27: VALIDACIÓN DEL MODELO LOGÍSTICO

In [13]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from IPython.display import display, Math
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [5]:
data = pd.read_csv('../../Recursos_curso/datasets/bank/bank.csv', sep = ';')

In [6]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


### Arreglar el dataframe

In [8]:
# Convertir en dummies

variables_categoricas = ["job", "marital", "education", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]

clean_data = data

for i in variables_categoricas:
    new_name = 'new_' + i
    new_dummy = pd.get_dummies(data[i], prefix = new_name)
    new_data = clean_data.join(new_dummy)
    clean_data = new_data

new_categories = clean_data.columns.values.tolist()
variables_no_categoricas = [i for i in new_categories if i not in variables_categoricas] 
variables_no_categoricas.remove('default')

new_dataframe = clean_data[variables_no_categoricas]
new_dataframe.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,new_month_oct,new_month_sep,new_day_of_week_fri,new_day_of_week_mon,new_day_of_week_thu,new_day_of_week_tue,new_day_of_week_wed,new_poutcome_failure,new_poutcome_nonexistent,new_poutcome_success
0,30,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,...,False,False,True,False,False,False,False,False,True,False
1,39,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,...,False,False,True,False,False,False,False,False,True,False
2,25,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,...,False,False,False,False,False,False,True,False,True,False
3,38,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,...,False,False,True,False,False,False,False,False,True,False
4,47,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,...,False,False,False,True,False,False,False,False,True,False


### Definir los mejores parametros para hacer la regresión logística

In [14]:
observaciones = ['y']
candidatos = new_dataframe.columns.values.tolist()
candidatos.remove('y')

numero_parametros = 12

parametros = RFE(LogisticRegression(), n_features_to_select = numero_parametros).fit(new_dataframe[candidatos], 
                                                                                     new_dataframe[observaciones].values.ravel())  

resultado_decision = list(zip(candidatos, parametros.support_, parametros.ranking_))

parametros_importantes = []
for i in range(len(resultado_decision)):
    if resultado_decision[i][1] == True:
        parametros_importantes.append(resultado_decision[i][0])

x_valores = new_dataframe[parametros_importantes].astype(float)
y_valores = new_dataframe['y']
for i in range(len(y_valores)):
    y_valores[i] = np.where(y_valores[i] == 'no', False, True)
    
y_valores = y_valores.astype(float)

C:\Users\Usuario\anaconda3\envs\curso_machine_learning\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Usuario\anaconda3\envs\curso_machine_learning\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

### Validación del modelo de regresión logística

In [ ]:
from sklearn.model_selection import train_test_split

In [18]:
# Lo primero que hay que hacer es dividir los datos en un conjunto de entrenamiento y otro de testing

x_entreno, x_test, y_entreno, y_test = train_test_split(x_valores, # esto es el dataframe con los parametros a usar
                                                        y_valores, # dataframe con las observaciones
                                                        test_size = 0.25, # porcentaje que queremos en el conjunto de testing
                                                        random_state = 0) 

In [21]:
print(f'Mientras que el conjunto de entrenamiento tiene {len(x_entreno)} observaciones, el conjunto de testing tiene {len(x_test)}.')

Mientras que el conjunto de entrenamiento tiene 3089 observaciones, el conjunto de testing tiene 1030.


In [22]:
# Construir el modelo de regresión logísitca

logistic_model = linear_model.LogisticRegression().fit(x_entreno, y_entreno)

In [23]:
# Predicción con el conjunto de test

prediccion = logistic_model.predict(x_test)

In [33]:
prediccion.tolist().count(1.0)

29

In [34]:
y_test.tolist().count(1.0)

101

In [36]:
# Comprobar como de buena es la regresión logística

from sklearn import metrics
metrics.accuracy_score(y_test, prediccion)

0.9048543689320389

### Validación Cruzada

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
scores = cross_val_score(linear_model.LogisticRegression(), x_valores, y_valores, scoring = "accuracy", cv = 10)

In [39]:
scores.mean()

0.9016748163371364